In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import tiktoken

import sys
import os
project_root = os.path.abspath('..')

if project_root not in sys.path:
    sys.path.append(project_root)
from model import GPTModel
from utils import create_dataloader_v1

## Wczytanie i konfiguracja modelu

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, #słownik
    "context_length": 256, #kontekst
    "emb_dim": 768, #osadzenia
    "n_heads": 12, #głowice uwagi
    "n_layers": 12, #warstwy
    "drop_rate": 0.1, #porzucenie
    "qkv_bias": False
}

In [3]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [4]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())


In [5]:
from utils import generate_text_simple

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Tekst wynikowy:\n", token_ids_to_text(token_ids, tokenizer))

Tekst wynikowy:
 Every effort moves you rentingetic minion mobilized Macicone warrantyuler anim supra


## Oblicznanie straty

### Ładowaie tekstu

In [7]:
file_path = "the-verdict.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read() 

In [8]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Liczba znaków:", total_characters)
print("Liczba tokenów:", total_tokens)

Liczba znaków: 20479
Liczba tokenów: 5145


### Podział danych

In [9]:
train_ratio = 0.9
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

In [10]:
train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_lenght=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_lenght=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [11]:
print("Szkoleniowy mechanizm ładujący:")
for x, y in train_loader:
    print(x.shape, y.shape)

Szkoleniowy mechanizm ładujący:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])


In [12]:
print("\nWalidacyjny mechanizm ładujący:")
for x, y in val_loader:
    print(x.shape, y.shape)


Walidacyjny mechanizm ładujący:
torch.Size([2, 256]) torch.Size([2, 256])


In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
        logits.flatten(0,1), target_batch.flatten()
    )
    return loss